In [0]:
from azure.eventhub.aio import EventHubProducerClient  # The package name suffixed with ".aio" for async
from azure.eventhub import EventData
import pandas as pd
import json
import asyncio
import time
import nest_asyncio
nest_asyncio.apply()

In [0]:
# Load Packages
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import yfinance as yf
yf.pdr_override()
from pandas_datareader import data as pdr
#import fix_yahoo_finance as yf

revenues de chaque cours 


df.display()

les données en streaming

In [0]:
tech_list = ['AAPL', 'GOOG', 'MSFT', 'AMZN','AAL' , 'ICE']
for stock in tech_list:
    globals()[stock] = yf.download(stock, start="2023-04-08", end="2023-04-11")
    
#Quick note: Using globals() is a sloppy way of setting the DataFrame names, but it's simple. Now we have our data, let's perform some basic data analysis and check our data.
company_list = [AAPL, GOOG, MSFT, AMZN, AAL , ICE]
company_name = ["AAPL", "GOOG", "MSFT", "AMZN","AAL" , "ICE"]

for company, com_name in zip(company_list, company_name):
    company["company_name"] = com_name
    
df1 = pd.concat(company_list, axis=0)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [0]:
df1= df1.reset_index()

In [0]:
def getclaims(data):
  json_data=json.loads(data.to_json(orient="records"))
  return json_data


In [0]:
# Assuming your DataFrame is called 'df' and the column name is 'timestamp_column'
df1['Date'] = df1['Date'].dt.strftime('%Y%m%d')

In [0]:
df1

,Date,Open,High,Low,Close,Adj Close,Volume,company_name
0,20230410,161.419998,162.029999,160.080002,162.029999,161.806183,47716900,AAPL
1,20230410,107.389999,107.970001,105.599998,106.949997,106.949997,19741500,GOOG
2,20230410,289.209991,289.600006,284.709991,289.390015,288.758759,23103000,MSFT
3,20230410,100.959999,102.199997,99.570000,102.169998,102.169998,37261200,AMZN
4,20230410,13.980000,14.170000,13.910000,14.150000,14.150000,18832300,AAL
5,20230410,107.529999,107.760002,105.739998,106.379997,106.379997,1780900,ICE


In [0]:
getclaims(df1)

Out[26]: [{'Date': '20230410',
  'Open': 161.4199981689,
  'High': 162.0299987793,
  'Low': 160.0800018311,
  'Close': 162.0299987793,
  'Adj Close': 161.8061828613,
  'Volume': 47716900,
  'company_name': 'AAPL'},
 {'Date': '20230410',
  'Open': 107.3899993896,
  'High': 107.9700012207,
  'Low': 105.5999984741,
  'Close': 106.9499969482,
  'Adj Close': 106.9499969482,
  'Volume': 19741500,
  'company_name': 'GOOG'},
 {'Date': '20230410',
  'Open': 289.2099914551,
  'High': 289.6000061035,
  'Low': 284.7099914551,
  'Close': 289.3900146484,
  'Adj Close': 288.7587585449,
  'Volume': 23103000,
  'company_name': 'MSFT'},
 {'Date': '20230410',
  'Open': 100.9599990845,
  'High': 102.1999969482,
  'Low': 99.5699996948,
  'Close': 102.1699981689,
  'Adj Close': 102.1699981689,
  'Volume': 37261200,
  'company_name': 'AMZN'},
 {'Date': '20230410',
  'Open': 13.9799995422,
  'High': 14.1700000763,
  'Low': 13.9099998474,
  'Close': 14.1499996185,
  'Adj Close': 14.1499996185,
  'Volume': 1883

In [0]:
#Configurations to send data to eventhub
connection_str = 'Endpoint=sb://evenetstream.servicebus.windows.net/;SharedAccessKeyName=streamdata;SharedAccessKey=Hfd1AYOfca/hX5J7/EbtPcTySlWOUFesH+AEhK64OQY=;EntityPath=eventhub1'
eventhub_name = 'eventhub1'

In [0]:
async def send_event_data_batch(producer,data):
    # Without specifying partition_id or partition_key
    # the events will be distributed to available partitions via round-robin.
    event_data_batch = await producer.create_batch()
    event_data_batch.add(EventData(data))
    await producer.send_batch(event_data_batch)

In [0]:
import time
async def run():

    producer = EventHubProducerClient.from_connection_string(connection_str, eventhub_name=eventhub_name)
    async with producer:
          for claim in getclaims(df1):
            time.sleep(2)
            await send_event_data_batch(producer,json.dumps(claim))
            print(claim)

In [0]:
loop = asyncio.get_event_loop()
start_time = time.time()
loop.run_until_complete(run())
print("Send messages in {} seconds.".format(time.time() - start_time))


{'Date': '20230410', 'Open': 161.4199981689, 'High': 162.0299987793, 'Low': 160.0800018311, 'Close': 162.0299987793, 'Adj Close': 161.8061828613, 'Volume': 47716900, 'company_name': 'AAPL'}
{'Date': '20230410', 'Open': 107.3899993896, 'High': 107.9700012207, 'Low': 105.5999984741, 'Close': 106.9499969482, 'Adj Close': 106.9499969482, 'Volume': 19741500, 'company_name': 'GOOG'}
{'Date': '20230410', 'Open': 289.2099914551, 'High': 289.6000061035, 'Low': 284.7099914551, 'Close': 289.3900146484, 'Adj Close': 288.7587585449, 'Volume': 23103000, 'company_name': 'MSFT'}
{'Date': '20230410', 'Open': 100.9599990845, 'High': 102.1999969482, 'Low': 99.5699996948, 'Close': 102.1699981689, 'Adj Close': 102.1699981689, 'Volume': 37261200, 'company_name': 'AMZN'}
{'Date': '20230410', 'Open': 13.9799995422, 'High': 14.1700000763, 'Low': 13.9099998474, 'Close': 14.1499996185, 'Adj Close': 14.1499996185, 'Volume': 18832300, 'company_name': 'AAL'}
{'Date': '20230410', 'Open': 107.5299987793, 'High': 107.